<a href="https://colab.research.google.com/github/wooje-jung/class-project/blob/main/%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8_%EA%B0%80%EA%B2%A9%EA%B3%BC_%EC%9D%B4%EC%8A%88%EC%99%80%EC%9D%98_%EA%B4%80%EB%A0%A8%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#0. 모듈 설치

In [1]:
!pip install pyupbit

In [2]:
!pip install selenium

#1. 비트코인 가격정보 수집


1.   수집대상 : 업비트 -> 비트코인 시가
2.   수집날짜 : 2017-09-25 ~ 2023.11.01



In [3]:
import pyupbit
import pandas as pd

In [4]:
pyupbit.get_tickers(fiat='KRW')

['KRW-BTC',
 'KRW-ETH',
 'KRW-NEO',
 'KRW-MTL',
 'KRW-XRP',
 'KRW-ETC',
 'KRW-SNT',
 'KRW-WAVES',
 'KRW-XEM',
 'KRW-QTUM',
 'KRW-LSK',
 'KRW-STEEM',
 'KRW-XLM',
 'KRW-ARDR',
 'KRW-ARK',
 'KRW-STORJ',
 'KRW-GRS',
 'KRW-ADA',
 'KRW-SBD',
 'KRW-POWR',
 'KRW-BTG',
 'KRW-ICX',
 'KRW-EOS',
 'KRW-TRX',
 'KRW-SC',
 'KRW-ONT',
 'KRW-ZIL',
 'KRW-POLYX',
 'KRW-ZRX',
 'KRW-LOOM',
 'KRW-BCH',
 'KRW-BAT',
 'KRW-IOST',
 'KRW-CVC',
 'KRW-IQ',
 'KRW-IOTA',
 'KRW-HIFI',
 'KRW-ONG',
 'KRW-GAS',
 'KRW-UPP',
 'KRW-ELF',
 'KRW-KNC',
 'KRW-BSV',
 'KRW-THETA',
 'KRW-QKC',
 'KRW-BTT',
 'KRW-MOC',
 'KRW-TFUEL',
 'KRW-MANA',
 'KRW-ANKR',
 'KRW-AERGO',
 'KRW-ATOM',
 'KRW-TT',
 'KRW-CRE',
 'KRW-MBL',
 'KRW-WAXP',
 'KRW-HBAR',
 'KRW-MED',
 'KRW-MLK',
 'KRW-STPT',
 'KRW-ORBS',
 'KRW-VET',
 'KRW-CHZ',
 'KRW-STMX',
 'KRW-DKA',
 'KRW-HIVE',
 'KRW-KAVA',
 'KRW-AHT',
 'KRW-LINK',
 'KRW-XTZ',
 'KRW-BORA',
 'KRW-JST',
 'KRW-CRO',
 'KRW-TON',
 'KRW-SXP',
 'KRW-HUNT',
 'KRW-PLA',
 'KRW-DOT',
 'KRW-MVL',
 'KRW-STRAX',
 'KRW-A

In [5]:
ticker = 'KRW-BTC'
interval = 'day'
to = '2023-11-01 09:00 '
count = 2292
data_price = pyupbit.get_ohlcv(ticker=ticker,interval=interval,to=to,count=count)
data_price = pd.DataFrame(data_price)
data_price['Price'] = 0
for i in range(0, 120):
    if data_price['open'][i] < data_price['open'][i+1]:
        data_price['Price'][i] = 1
    else:
        data_price['Price'][i] = 0
open_data_price = [['open','Price']]


<ipython-input-5-7e7fb7578394>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_price['Price'][i] = 1
<ipython-input-5-7e7fb7578394>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_price['Price'][i] = 0


In [6]:
data_price
open_data_price = data_price[['open','Price']]
open_data_price

,open,Price
2017-09-25 09:00:00,4201000.0,1
2017-09-26 09:00:00,4317000.0,1
2017-09-27 09:00:00,4322000.0,1
2017-09-28 09:00:00,4657000.0,0
2017-09-29 09:00:00,4586000.0,1
...,...,...
2023-10-28 09:00:00,45980000.0,0
2023-10-29 09:00:00,46316000.0,0
2023-10-30 09:00:00,46825000.0,0
2023-10-31 09:00:00,46734000.0,0


#2.뉴스 정보 크롤링(비트코인 관련 뉴스)

1.   수집대상 : google비트코인 관련뉴스 -> 제목

2.   수집날짜 : 2017-09-24 ~ 2023.10.31






In [23]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from tqdm.notebook import tqdm
from datetime import datetime, timedelta
import re
import requests

In [71]:
def generate_date_urls(start_date, end_date):
    url_result = {}

    current_date = start_date
    while current_date <= end_date:
        y, m, d = current_date.year, current_date.month, current_date.day
        month_day = f"{y}-{m:02d}-{d:02d}"

        # day_range: 각 월의 마지막 날
        if m in [1, 3, 5, 7, 8, 10, 12]:
            day_range = 31
        elif m == 2:
            if (y % 4 == 0 and y % 100 != 0) or (y % 400 == 0):
                day_range = 29  # 윤년
            else:
                day_range = 28
        else:
            day_range = 30

        for j in range(d, day_range + 1):
            day = j
            month_day = f"{y}-{m:02d}-{day:02d}"
            duration = f"%2Ccd_min%3A{m}%2F{day}%2F{y}%2Ccd_max%3A{m}%2F{day}%2F{y}"
            base_url = f"https://www.google.com/search?q=bitcoin&biw=1005&bih=842&source=lnt&tbs=cdr%3A1{duration}&tbm=nws"
            url_result[month_day] = base_url

        # 다음 달로 이동
        if m == 12:
            current_date = datetime(y + 1, 1, 1)
        else:
            current_date = datetime(y, m, day_range) + timedelta(days=1)

    return url_result

# Set start and end dates
start_date = datetime(2017, 9,30)
end_date = datetime(2023, 10,31)

# Generate date URLs
url_dict = generate_date_urls(start_date, end_date)

# Display the result
for date, url in url_dict.items():
    print(f"{date}: {url}")

2017-09-30: https://www.google.com/search?q=bitcoin&biw=1005&bih=842&source=lnt&tbs=cdr%3A1%2Ccd_min%3A9%2F30%2F2017%2Ccd_max%3A9%2F30%2F2017&tbm=nws
2017-10-01: https://www.google.com/search?q=bitcoin&biw=1005&bih=842&source=lnt&tbs=cdr%3A1%2Ccd_min%3A10%2F1%2F2017%2Ccd_max%3A10%2F1%2F2017&tbm=nws
2017-10-02: https://www.google.com/search?q=bitcoin&biw=1005&bih=842&source=lnt&tbs=cdr%3A1%2Ccd_min%3A10%2F2%2F2017%2Ccd_max%3A10%2F2%2F2017&tbm=nws
2017-10-03: https://www.google.com/search?q=bitcoin&biw=1005&bih=842&source=lnt&tbs=cdr%3A1%2Ccd_min%3A10%2F3%2F2017%2Ccd_max%3A10%2F3%2F2017&tbm=nws
2017-10-04: https://www.google.com/search?q=bitcoin&biw=1005&bih=842&source=lnt&tbs=cdr%3A1%2Ccd_min%3A10%2F4%2F2017%2Ccd_max%3A10%2F4%2F2017&tbm=nws
2017-10-05: https://www.google.com/search?q=bitcoin&biw=1005&bih=842&source=lnt&tbs=cdr%3A1%2Ccd_min%3A10%2F5%2F2017%2Ccd_max%3A10%2F5%2F2017&tbm=nws
2017-10-06: https://www.google.com/search?q=bitcoin&biw=1005&bih=842&source=lnt&tbs=cdr%3A1%2Ccd_min

In [ ]:
import requests
def scrape_news_titles(url_dict):
    news_titles = {}
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'}

    for date, url in url_dict.items():
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        resultpage = soup.select_one("div#search")
        link = resultpage.select("a")
        day_titles = []
        for li in link:
            pageUrl = li['href']
            try:
                pageCrawling = requests.get(pageUrl, headers=headers)
                pagesource = BeautifulSoup(pageCrawling.text, 'html.parser')
                title = pagesource.select_one("title").text
                day_titles.append(title)
            except:
                continue
        news_titles[date] = day_titles
    return news_titles

result = scrape_news_titles(url_dict)

In [68]:
result

{'2017-10-01': ['What Is Bitcoin, and How Does It Work? - The New York Times',
  'Why China dominates bitcoin and how it may control the blockchain technology behind it',
  'Eminem songs, Bitcoin are hyped investments, but what are the risks?',
  "Guns & Crypto: How Bitcoin Is Helping Keep Cody Wilson's Anarchist Dream Alive - CoinDesk",
  "Creators of Disney's Blockchain Build Commercial Platform on Dragonchain With ICO",
  'Droppet ut av skolen som 17-åring – grunnla kryptovaluta verdt 13,5 milliarder – E24',
  'Tudo sobre a moeda virtual Bitcoin',
  'Bitcoin: una nación flotante',
  'CRYPTO20 - An Autonomous, Tokenized, Crypto Index Fund | Bitcoinist.com',
  'Bitcoinbányászás napenergiával? - Nem rossz, de a negatív áramár mindent visz - Portfolio.hu'],
 '2017-10-02': ['bitcoinmagazine.com',
  'Police Confirm North Korean Connection in Bitcoin Exchange Phishing - CoinDesk',
  'The New Internet Bubble: Meet The Young Entrepreneurs Riding Out The Crypto-Craziness',
  '6 Unique ICOs to

In [69]:
result_df = pd.DataFrame(list(result.items()), columns=['Date', 'contents'])

In [70]:
result_df

,Date,contents
0,2017-10-01,"[What Is Bitcoin, and How Does It Work? - The ..."
1,2017-10-02,"[bitcoinmagazine.com, Police Confirm North Kor..."
2,2017-10-03,[US government misses out on $600 million payd...
3,2017-10-04,[Bitcoin may become 'expected' part of a portf...
4,2017-10-05,[Cryptocurrency in India: Usage and Regulation...
5,2017-10-06,"[4 Things You Should Ask A Bitcoin Broker, Cas..."
6,2017-10-07,[Bitcoin Price Is Rising but It Could Be Usurp...
7,2017-10-08,[How Blockchain Is Changing the Energy Industr...
8,2017-10-09,[Markets Update: The Bitcoin Price Rocket Blas...
9,2017-10-10,"[Harvard Professor: ""In the Long Run, the Pric..."
